In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
import matplotlib.ticker as plticker
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression as logreg

In [4]:
ipl_pred = pd.read_csv('/home/jay/Desktop/IPL/IPl_history.csv')
results = pd.read_csv('/home/jay/Desktop/IPL/ipl_results.csv')

In [153]:
results

,Team 1,Team 2,Winner,Margin,Ground,Match Date,Scorecard
0,RCB,KKR,KKR,140 runs,Bengaluru,"Apr 18, 2008",T20
1,KXIP,CSK,CSK,33 runs,Mohali,"Apr 19, 2008",T20
2,DC,RR,DC,9 wickets,Delhi,"Apr 19, 2008",T20
3,MI,RCB,RCB,5 wickets,Mumbai,"Apr 20, 2008",T20
4,RR,KXIP,RR,6 wickets,Jaipur,"Apr 21, 2008",T20
...,...,...,...,...,...,...,...
559,MI,KKR,MI,9 wickets,Mumbai,"May 5, 2019",T20
560,MI,CSK,MI,6 wickets,Chennai,"May 7, 2019",T20
561,DC,SRH,DC,2 wickets,Visakhapatnam,"May 8, 2019",T20
562,CSK,DC,CSK,6 wickets,Visakhapatnam,"May 10, 2019",T20


In [155]:
results.rename(columns= {'Team 1':'Team_1','Team 2':'Team_2'},inplace=True)

In [156]:
results

,Team_1,Team_2,Winner,Margin,Ground,Match Date,Scorecard
0,RCB,KKR,KKR,140 runs,Bengaluru,"Apr 18, 2008",T20
1,KXIP,CSK,CSK,33 runs,Mohali,"Apr 19, 2008",T20
2,DC,RR,DC,9 wickets,Delhi,"Apr 19, 2008",T20
3,MI,RCB,RCB,5 wickets,Mumbai,"Apr 20, 2008",T20
4,RR,KXIP,RR,6 wickets,Jaipur,"Apr 21, 2008",T20
...,...,...,...,...,...,...,...
559,MI,KKR,MI,9 wickets,Mumbai,"May 5, 2019",T20
560,MI,CSK,MI,6 wickets,Chennai,"May 7, 2019",T20
561,DC,SRH,DC,2 wickets,Visakhapatnam,"May 8, 2019",T20
562,CSK,DC,CSK,6 wickets,Visakhapatnam,"May 10, 2019",T20


In [157]:
results=results[['Match Date','Team_1','Team_2','Winner','Margin','Ground']]

In [158]:
results.head()

,Match Date,Team_1,Team_2,Winner,Margin,Ground
0,"Apr 18, 2008",RCB,KKR,KKR,140 runs,Bengaluru
1,"Apr 19, 2008",KXIP,CSK,CSK,33 runs,Mohali
2,"Apr 19, 2008",DC,RR,DC,9 wickets,Delhi
3,"Apr 20, 2008",MI,RCB,RCB,5 wickets,Mumbai
4,"Apr 21, 2008",RR,KXIP,RR,6 wickets,Jaipur


In [159]:
df = results[(results['Team_1'] == 'RR') | (results['Team_2'] == 'RR')]

In [160]:
RR=df.iloc[:]
RR.head()

,Match Date,Team_1,Team_2,Winner,Margin,Ground
2,"Apr 19, 2008",DC,RR,DC,9 wickets,Delhi
4,"Apr 21, 2008",RR,KXIP,RR,6 wickets,Jaipur
8,"Apr 26, 2008",RCB,RR,RR,7 wickets,Bengaluru
13,"May 1, 2008",RR,KKR,RR,45 runs,Jaipur
17,"May 4, 2008",RR,CSK,RR,8 wickets,Jaipur


In [161]:
year = []
for row in RR['Match Date']:
    year.append(int(row[8:]))  #date mai 7th char ke aage year hai
RR ['match_year']= year
RR_2008 = RR[RR.match_year >= 2008]
RR_2008.count()

Match Date    95
Team_1        95
Team_2        95
Winner        95
Margin        93
Ground        95
match_year    95
dtype: int64

In [162]:
ipl_teams = ['RR', 'CSK', 'KKR', 'RCB', 
            'DC', 'Sunrisers', 'KXIP', 'MI']
df_teams_1 = results[results['Team_1'].isin(ipl_teams)]
df_teams_2 = results[results['Team_2'].isin(ipl_teams)]
df_teams = pd.concat((df_teams_1, df_teams_2))
df_teams.drop_duplicates() 
df_teams.count()

Match Date    1033
Team_1        1033
Team_2        1033
Winner        1033
Margin        1025
Ground        1033
dtype: int64

In [163]:
df_teams_2008 = df_teams.drop(['Match Date','Margin', 'Ground'], axis=1)  #drop unrequired columns
df_teams_2008.head()

,Team_1,Team_2,Winner
0,RCB,KKR,KKR
1,KXIP,CSK,CSK
2,DC,RR,DC
3,MI,RCB,RCB
4,RR,KXIP,RR


In [164]:
df_teams_2008 = df_teams_2008.reset_index(drop=True)
df_teams_2008.loc[df_teams_2008.Winner == df_teams_2008.Team_1,'winning_team']=1
df_teams_2008.loc[df_teams_2008.Winner == df_teams_2008.Team_2, 'winning_team']=2
df_teams_2008 = df_teams_2008.drop(['winning_team'], axis=1)

df_teams_2008.head()

,Team_1,Team_2,Winner
0,RCB,KKR,KKR
1,KXIP,CSK,CSK
2,DC,RR,DC
3,MI,RCB,RCB
4,RR,KXIP,RR


In [165]:
final = pd.get_dummies(df_teams_2008, prefix=['Team_1', 'Team_2'], columns=['Team_1', 'Team_2'])

X = final.drop(['Winner'], axis=1)
y = final["Winner"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)


In [166]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, max_depth=20,random_state=0) 
rf.fit(X_train, y_train)
score = rf.score(X_train, y_train)
score2 = rf.score(X_test, y_test)
print("Training set accuracy: ", '%.3f'%(score))
print("Test set accuracy: ", '%.3f'%(score2))


Training set accuracy:  0.645
Test set accuracy:  0.590


In [167]:
ranking = pd.read_csv('/home/jay/Desktop/IPL/Ipl_rankings.csv') 
fixtures2 = pd.read_csv('/home/jay/Desktop/IPL/IPL_Schedule.csv')
pred_set = []

In [168]:
fixtures2.rename(columns= {'Team 1':'Team_1','Team 2':'Team_2'},inplace=True)
ranking.rename(columns= {'Rankings':'Position'},inplace=True)

In [169]:
ranking

,Team,Total Matches Won,Win %,Position
0,MI,107,57.75,1
1,CSK,100,61.28,2
2,KKR,92,52.52,3
3,RCB,83,47.45,4
4,KXIP,80,46.02,5
5,DC,76,44.00,6
6,RR,73,51.37,7
7,SRH,57,53.70,8


In [170]:
fixtures2.tail()

,Match No.,Date,Team_1,Team_2,Ground
55,56,May 17th 2020,RCB,MI,Bengaluru
56,57,May 18th 2020,TBA,TBA,Jaipur
57,58,May 19th 2020,TBA,TBA,Chennai
58,59,May 20th 2020,TBA,TBA,Delhi
59,60,May 21th 2020,TBA,TBA,Mumbai


In [171]:
fixtures2.insert(1, 'first_position', fixtures2['Team_1'].map(ranking.set_index('Team')['Position']))
fixtures2.insert(2, 'second_position', fixtures2['Team_2'].map(ranking.set_index('Team')['Position']))

fixtures2 = fixtures2.iloc[:56, :]
fixtures2.tail()


,Match No.,first_position,second_position,Date,Team_1,Team_2,Ground
51,52,6.0,7.0,May 13th 2020,DC,RR,Delhi
52,53,4.0,2.0,May 14th 2020,RCB,CSK,Bengaluru
53,54,3.0,8.0,May 15th 2020,KKR,SRH,Kolkata
54,55,5.0,6.0,May 16th 2020,KXIP,DC,Mohali
55,56,4.0,1.0,May 17th 2020,RCB,MI,Bengaluru


In [172]:
for index, row in fixtures2.iterrows():
    if row['first_position'] < row['second_position']:
        pred_set.append({'Team_1': row['Team_1'], 'Team_2': row['Team_2'], 'winning_team': None})
    else:
        pred_set.append({'Team_1': row['Team_2'], 'Team_2': row['Team_1'], 'winning_team': None})
        
pred_set = pd.DataFrame(pred_set)
backup_pred_set = pred_set
pred_set.head()


,Team_1,Team_2,winning_team
0,MI,CSK,None
1,KXIP,DC,None
2,KKR,RCB,None
3,MI,SRH,None
4,CSK,RR,None


In [173]:
pred_set = pd.get_dummies(pred_set, prefix=['Team_1', 'Team_2'], columns=['Team_1', 'Team_2'])

missing_cols = set(final.columns) - set(pred_set.columns)
for c in missing_cols:
    pred_set[c] = 0
pred_set = pred_set[final.columns]


pred_set = pred_set.drop(['Winner'], axis=1)
pred_set.head()


,Team_1_CSK,Team_1_DC,Team_1_KKR,Team_1_KXIP,Team_1_MI,Team_1_RCB,Team_1_RR,Team_1_SRH,Team_2_CSK,Team_2_DC,Team_2_KKR,Team_2_KXIP,Team_2_MI,Team_2_RCB,Team_2_RR,Team_2_SRH
0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [130]:
predictions = rf.predict(pred_set)
for i in range(fixtures2.shape[0]):
    print(backup_pred_set.iloc[i, 1] + " and " + backup_pred_set.iloc[i, 0])
    if predictions[i] == 1:
        print("Winner: " + backup_pred_set.iloc[i, 1])
    
    else:
        print("Winner: " + backup_pred_set.iloc[i, 0])
    print("")

CSK and MI
Winner: MI

DC and KXIP
Winner: KXIP

RCB and KKR
Winner: KKR

SRH and MI
Winner: MI

RR and CSK
Winner: CSK

DC and KKR
Winner: KKR

SRH and KXIP
Winner: KXIP

RCB and MI
Winner: MI

RR and DC
Winner: DC

KKR and CSK
Winner: CSK

SRH and RCB
Winner: RCB

KXIP and MI
Winner: MI

RR and KKR
Winner: KKR

DC and RCB
Winner: RCB

KXIP and CSK
Winner: CSK

SRH and RR
Winner: RR

KKR and MI
Winner: MI

DC and CSK
Winner: CSK

KXIP and RCB
Winner: RCB

RR and MI
Winner: MI

SRH and KKR
Winner: KKR

KXIP and CSK
Winner: CSK

RR and RCB
Winner: RCB

DC and KKR
Winner: KKR

SRH and CSK
Winner: CSK

KXIP and MI
Winner: MI

SRH and RR
Winner: RR

DC and RCB
Winner: RCB

KXIP and KKR
Winner: KKR

CSK and MI
Winner: MI

RR and RCB
Winner: RCB

KXIP and KKR
Winner: KKR

SRH and DC
Winner: DC

RCB and CSK
Winner: CSK

KKR and MI
Winner: MI

RR and KXIP
Winner: KXIP

SRH and CSK
Winner: CSK

DC and MI
Winner: MI

RR and KKR
Winner: KKR

KXIP and RCB
Winner: RCB

SRH and DC
Winner: DC

RR and

In [174]:
semi=[('MI','RCB'),('CSK','KKR')]

In [175]:
def clean_and_predict(matches, ranking, final, logreg):

    # Initialization of auxiliary list for data cleaning
    positions = []

    # Loop to retrieve each team's position according to ICC ranking
    for match in matches:
        positions.append(ranking.loc[ranking['Team'] == match[0],'Position'].iloc[0])
        positions.append(ranking.loc[ranking['Team'] == match[1],'Position'].iloc[0])
    
    # Creating the DataFrame for prediction
    pred_set = []

    # Initializing iterators for while loop
    i = 0
    j = 0

    # 'i' will be the iterator for the 'positions' list, and 'j' for the list of matches (list of tuples)
    while i < len(positions):
        dict1 = {}

        # If position of first team is better then this team will be the 'Team_1' team, and vice-versa
        if positions[i] < positions[i + 1]:
            dict1.update({'Team_1': matches[j][0], 'Team_2': matches[j][1]})
        else:
            dict1.update({'Team_1': matches[j][1], 'Team_2': matches[j][0]})

        # Append updated dictionary to the list, that will later be converted into a DataFrame
        pred_set.append(dict1)
        i += 2
        j += 1
        
        # Convert list into DataFrame
    pred_set = pd.DataFrame(pred_set)
    backup_pred_set = pred_set

    # Get dummy variables and drop winning_team column
    pred_set = pd.get_dummies(pred_set, prefix=['Team_1', 'Team_2'], columns=['Team_1', 'Team_2'])

    # Add missing columns compared to the model's training dataset
    missing_cols2 = set(final.columns) - set(pred_set.columns)
    for c in missing_cols2:
        pred_set[c] = 0
    pred_set = pred_set[final.columns]

    pred_set = pred_set.drop(['Winner'], axis=1)

    # Predict!
    predictions = logreg.predict(pred_set)
    for i in range(len(pred_set)):
        print(backup_pred_set.iloc[i, 1] + " and " + backup_pred_set.iloc[i, 0])
        if predictions[i] == 1:
            print("Winner: " + backup_pred_set.iloc[i, 1])
        else:
            print("Winner: " + backup_pred_set.iloc[i, 0])
        print("")


In [176]:
clean_and_predict(semi, ranking, final, rf)

RCB and MI
Winner: MI

KKR and CSK
Winner: CSK



In [177]:
finale=[('MI','CSK')]

In [178]:
clean_and_predict(finale, ranking, final, rf)

CSK and MI
Winner: MI

